In [1]:
import sys
sys.path.append("../")
from pathlib import Path

from tqdm import tqdm
import pandas as pd
import numpy as np

from const import gnps, mona
from utils import embedding, most_similar, search_with_embedding, get_smiles, cosine_similarity

/data/xp/Anaconda/envs/py312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
spectra_paths = {
    "gnps":{
        "orbitrap": {
            "train": (gnps.ORBITRAP_TRAIN_QUERY, gnps.ORBITRAP_TEST_REF),
            "test": (gnps.ORBITRAP_TEST_QUERY, gnps.ORBITRAP_TEST_REF)
        },
        "qtof": {
            "test": (gnps.QTOF_TEST_QUERY, gnps.QTOF_TEST_REF)
        },
        "other": {
            "test": (gnps.OTHER_TEST_QUERY, gnps.OTHER_TEST_REF)
        }
    }
}

In [3]:
show_progress_bar = False
replica_suffix = "-replication-{}"
k_metric = [5, 1, 10]
batch_size = None

In [4]:
replica_df_seq = []

for i in tqdm(range(10)):
    df_seq = []
    for db, db_metadata in spectra_paths.items():
        for desc, path_metadata in db_metadata.items():
            for info, paths in path_metadata.items():
                print("-" * 40, f"{db}-{desc}-{info}", "-" * 40)
                query_path, ref_path = paths
                query_path = query_path.with_stem(query_path.stem + replica_suffix.format(i + 1))
                ref_path = ref_path.with_stem(ref_path.stem + replica_suffix.format(i + 1))
                if db == "gnps" and desc == "orbitrap":
                    if info == "train":
                        query_path = gnps.ORBITRAP_TRAIN_QUERY
                    
                    train_ref_spectra, train_ref_embedding = embedding(
                        str(gnps.ORBITRAP_TRAIN_REF.with_suffix(".mgf")),
                        show_progress_bar
                    )
                    ref_spectra, ref_embedding = embedding(
                        str(ref_path.with_suffix(".mgf")),
                        show_progress_bar
                    )
                    query_spectra, query_embedding = embedding(
                        str(query_path.with_suffix(".mgf")),
                        show_progress_bar
                    )
                    ref_spectra = np.hstack((train_ref_spectra, ref_spectra))
                    ref_embedding = np.concatenate((train_ref_embedding, ref_embedding))

                else:
                    ref_spectra, ref_embedding = embedding(
                        str(ref_path.with_suffix(".mgf")),
                        show_progress_bar,
                    )
                    query_spectra, query_embedding = embedding(
                        str(query_path.with_suffix(".mgf")),
                        show_progress_bar
                    )

                ref_smiles = get_smiles(ref_spectra)
                query_smiles = get_smiles(query_spectra)
                df = search_with_embedding(
                f"{db}-{desc}-{info}", k_metric,
                query_embedding, ref_embedding,
                query_smiles, ref_smiles,
                show_progress_bar, batch_size
                )
                df_seq.append(df)

    df = pd.concat(df_seq, axis=0)
    print(df)
    replica_df_seq.append(df)

  0%|          | 0/10 [00:00<?, ?it/s]

---------------------------------------- gnps-orbitrap-train ----------------------------------------
---------------------------------------- gnps-orbitrap-test ----------------------------------------
---------------------------------------- gnps-qtof-test ----------------------------------------
---------------------------------------- gnps-other-test ----------------------------------------


 10%|█         | 1/10 [09:29<1:25:24, 569.38s/it]

                         top1      top5     top10
gnps-orbitrap-train  0.711283  0.835498  0.868048
gnps-orbitrap-test   0.720641  0.836892  0.866548
gnps-qtof-test       0.524468  0.708245  0.747340
gnps-other-test      0.746706  0.887614  0.914432
---------------------------------------- gnps-orbitrap-train ----------------------------------------
---------------------------------------- gnps-orbitrap-test ----------------------------------------
---------------------------------------- gnps-qtof-test ----------------------------------------
---------------------------------------- gnps-other-test ----------------------------------------


 20%|██        | 2/10 [19:00<1:16:02, 570.30s/it]

...

                         top1      top5     top10
gnps-orbitrap-train  0.712305  0.835207  0.867611
gnps-orbitrap-test   0.711744  0.829775  0.862989
gnps-qtof-test       0.522340  0.703723  0.743750
gnps-other-test      0.743916  0.888389  0.915517
---------------------------------------- gnps-orbitrap-train ----------------------------------------
---------------------------------------- gnps-orbitrap-test ----------------------------------------
---------------------------------------- gnps-qtof-test ----------------------------------------
---------------------------------------- gnps-other-test ----------------------------------------


 90%|█████████ | 9/10 [1:24:41<09:24, 564.96s/it]

                         top1      top5     top10
gnps-orbitrap-train  0.711867  0.835353  0.867611
gnps-orbitrap-test   0.714116  0.837485  0.864769
gnps-qtof-test       0.525798  0.702261  0.741489
gnps-other-test      0.741280  0.887769  0.913347
---------------------------------------- gnps-orbitrap-train ----------------------------------------
---------------------------------------- gnps-orbitrap-test ----------------------------------------
---------------------------------------- gnps-qtof-test ----------------------------------------
---------------------------------------- gnps-other-test ----------------------------------------


100%|██████████| 10/10 [1:34:07<00:00, 564.77s/it]

                         top1      top5     top10
gnps-orbitrap-train  0.712597  0.835498  0.867902
gnps-orbitrap-test   0.716489  0.844603  0.874259
gnps-qtof-test       0.523803  0.701729  0.739761
gnps-other-test      0.739730  0.886839  0.913192


In [5]:
data = []
indices = replica_df_seq[0].index
columns = replica_df_seq[0].columns
for item in replica_df_seq:
    data.append([item.values])

In [6]:
data = np.concatenate(data, axis=0)
np.set_printoptions(precision=4, suppress=True)
np.mean(data, axis=0) * 100, np.std(data, axis=0) * 100

(array([[71.2042, 83.5513, 86.7815],
        [71.9039, 84.1934, 87.0225],
        [52.4269, 70.3989, 74.246 ],
        [74.3931, 88.7242, 91.3796]]),
 array([[0.0396, 0.0394, 0.0334],
        [0.5775, 0.7069, 0.6157],
        [0.2881, 0.2814, 0.3022],
        [0.4069, 0.2148, 0.1557]]))

In [7]:
pd.set_option('display.precision', 4)
mean_df = pd.DataFrame(np.mean(data, axis=0) * 100, index=indices, columns=columns)
std_df = pd.DataFrame(np.std(data, axis=0) * 100, index=indices, columns=columns)

In [8]:
mean_df.to_csv(f"./mean.tsv", sep='\t')
std_df.to_csv(f"./std.tsv", sep='\t')

In [2]:
show_progress_bar = False
k_metric = [5, 1, 10]
batch_size = None

dir_ = Path("/data1/xp/data/MSBert/MTBLS1572")
query_path = dir_ / "query.npy"
ref_path = dir_ / "ref.npy"

query_spectra, query_embedding = embedding(query_path.with_suffix(".mgf"), show_progress_bar)
ref_spectra, ref_embedding = embedding(ref_path.with_suffix(".mgf"), show_progress_bar)
query_smiles = get_smiles(query_spectra)
ref_smiles = get_smiles(ref_spectra)

search_with_embedding(
    "MTBLS1572", k_metric,
    query_embedding, ref_embedding,
    query_smiles, ref_smiles,
    show_progress_bar, batch_size
)

,top1,top5,top10
MTBLS1572,0.857143,1.0,1.0


In [4]:
cosine_score = cosine_similarity(
    query_embedding, ref_embedding
)
for i, j in enumerate(np.argmax(cosine_score, axis=1)):
    if i != j:
        print(f"{i}-th answer is [{ref_spectra[i].get("compound_name")}] but get {j}-th [{ref_spectra[j].get("compound_name")}]")

4-th answer is [Butyryl-L-carnitine] but get 11-th [Isovaleryl-L-carnitine]
17-th answer is [Pantothenic acid] but get 19-th [Propionyl-L-carnitine]
18-th answer is [Proline] but get 16-th [Ornithine]


In [6]:
cosine_score[4][4], cosine_score[4][11]

(0.4315794722035844, 0.6193784627802925)

In [2]:
show_progress_bar = True

orbitrap_ref_spectra, orbitrap_ref_embedding = embedding(
    str(mona.ORBITRAP_COMMON.with_suffix(".mgf")), 
    show_progress_bar
)
orbitrap_query_spectra, orbitrap_query_embedding = embedding(
    str(mona.ORBITRAP_UNIQUE.with_suffix(".mgf")), 
    show_progress_bar
)
qtof_ref_spectra, qtof_ref_embedding = embedding(
    str(mona.ORBITRAP_COMMON.with_suffix(".mgf")), 
    show_progress_bar
)
qtof_query_spectra, qtof_query_embedding = embedding(
    str(mona.QTOF_UNIQUE.with_suffix(".mgf")), 
    show_progress_bar
)

Computing DreaMS embedding: 100%|██████████| 674/674 [00:00<00:00, 861.37it/s]


In [3]:
orbitrap_score, orbitrap_indices = most_similar(orbitrap_query_embedding, orbitrap_ref_embedding, 512, show_progress_bar)
qtof_score, qtof_indices = most_similar(qtof_query_embedding, qtof_ref_embedding, 512, show_progress_bar)

processing: 100%|██████████| 2/2 [00:00<00:00, 16.95it/s]


In [4]:
dir = Path("/data1/xp/code/specEmbedding/score_distribution/compound search/")
orbitrap_dir = dir / "orbitrap"
qtof_dir = dir / "qtof"

orbitrap_dir.mkdir(exist_ok=True, parents=True)
qtof_dir.mkdir(exist_ok=True, parents=True)

np.save(orbitrap_dir / "DreaMS_Score.npy", orbitrap_score)
np.save(orbitrap_dir / "DreaMS_Indices.npy", orbitrap_indices)
np.save(qtof_dir / "DreaMS_Score.npy", qtof_score)
np.save(qtof_dir / "DreaMS_Indices.npy", qtof_indices)

In [13]:
show_progress_bar = True
k_metric = [5, 1, 10]
batch_size = None

query_spectra, query_embedding = embedding(mona.ORBITRAP_COMMON.with_suffix(".mgf"), show_progress_bar)
ref_spectra, ref_embedding = embedding(gnps.ORBITRAP_ALL.with_suffix(".mgf"), show_progress_bar)
query_smiles = get_smiles(query_spectra)
ref_smiles = get_smiles(ref_spectra)

search_with_embedding(
    "Orbitrap Common", k_metric,
    query_embedding, ref_embedding,
    query_smiles, ref_smiles,
    show_progress_bar, batch_size
)

calculate hit and recall count: 100%|██████████| 1/1 [00:05<00:00,  5.77s/it]


,top1,top5,top10
Orbitrap Common,0.8178,0.878,0.9043


In [14]:
show_progress_bar = True
k_metric = [5, 1, 10]
batch_size = None

query_spectra, query_embedding = embedding(mona.QTOF_COMMON.with_suffix(".mgf"), show_progress_bar)
ref_spectra, ref_embedding = embedding(gnps.QTOF_ALL.with_suffix(".mgf"), show_progress_bar)
query_smiles = get_smiles(query_spectra)
ref_smiles = get_smiles(ref_spectra)

search_with_embedding(
    "QTOF Common", k_metric,
    query_embedding, ref_embedding,
    query_smiles, ref_smiles,
    show_progress_bar, batch_size
)

calculate hit and recall count: 100%|██████████| 1/1 [00:01<00:00,  1.64s/it]


,top1,top5,top10
QTOF Common,0.9873,0.9953,0.997
